### ***Theory of GAs***

The first appearence of genetic algorithms (GAs) was on J. D. Bagley's thesis *"The behaviour of adaptive systems which employ genetic and correlative algorithm"*. He described them like this: *"[...] genetic algorithms are simulations of eveolution, of what kind ever. In most cases, however, genetic algorithms are nothing else than probabilistic optimisation methods which are based on the principles of evolutions"*.

In general, the problem GAs are used to solve is to find an $x_\in X$ such that an arbitrary $f:X \longrightarrow \mathbb{R}$ is maximal, i.e. $f(x_0)=\max_{x\in X}f(x)$. Depending on the actual problem, it can be quite difficult to obtain such point and it might be sufficent to have a local maximum or to be as close as possible to the global maximum. The function $f$ is called *"fitness"* becuase it assigns values to individuals $x\in X$ so that we can compare them. Reproduction and adaption are carried out at genetic information level, so GAs do not operate on the values of the search space $X$, but on some coded versions of them (like strings or arrays). In our case

Assume $S$ to be a set of strings (or arrays) in general with underlyning grammar and let $X$ be the search space of the above optimization problem, then the function:
$$c: X \longrightarrow S$$
is called a *conding function*; conversely:
$$c^{-1}: S \rightarrow X$$
is called a *decoding function*.
In practice, coding and decoding function have to be specified according to the needs of the actual problem.

The transition from one generation to the next consists of four basic coponents:
1. ***selection***: choose the individuals to be reproduced according to their fitness values.
2. ***crossover***: merge the genetic information of two individuals (i.e. their chromosomes); if implemented correctly, good paranets produce good children.
3. ***mutation***: in GA, mutation is realized as a random deformation of the chromosone array with a certain probability. The positive effects are the preservation of genetic diversity and avoiding of local maxima.
4. ***sampling***: it's the procedure that computes a new generation from the previous one and its offsprings.

Note: normal genetic algorithms do not use any auxiliary information about the objective function such as its gradient, so they can be applied to any kind of continuous or discrete optimization problem. Moreover, since they work on the whole population and not on a single point at time (like GD), they are more robust and have more chance to find the global optimum.

---

Note: the paper doesn't describe in depth all the parts of the notebook and the design choices made; we have made them using what we learnt, maybe other choices could be more effective.

The starting dataset is made of $F$ features, so $X$ will be the *power set* of $F$: $$X=\mathcal{P}(F)=\{A\,|\,A\subseteq F\}$$
Chromosomes are represented as array of length $|F|$ ($b = \{0, 1\}^{|F|}$) where $i\text{-th}$ gene represents wheter feature $f_i$ is considered by the individual. Coding and decoding functions come naturally:
$$
c(A) = (b_1, b_2, \dots, b_{|F|}), \quad \text{where } b_i =
\begin{cases}
1, & \text{if } f_i \in A, \\
0, & \text{otherwise.}
\end{cases}
$$

$$
c^{-1}(b) = \{ f_i \in F \mid b_i = 1 \}
$$

Authors suggested to take as fiteness function $\phi$ the accuracy.  

At each timestamp $t$, we'll consider a fixed size population with $m$ individuals:
$$\mathcal{B}_t=(b_{1, t}, b_{2, t}, \dots, b_{m, t})$$

The algorithm developed in this notebook is the following:
$$
\begin{aligned}
& \textbf{GA}(\phi_m : \text{min fitness to reach}\\
&\quad \quad      m: \text{population size},\\
&\quad\quad      F: \text{initial set of features}, \\
&\quad\quad      k: \text{total number of iterations},\\
&\quad\quad      p_m : \text{probability of mutation},\\
&\quad\quad      p_c : \text{probability of crossover},\\
&\quad\quad      n: \text{number of crossover points},\\
&\quad\quad      \phi_T: \text{threshold on the increment of fitness},\\
&\quad\quad      X_{train}: \text{matrix with training samples}, \\
&\quad\quad      y_{train}: \text{vector with training labels}, \\
&\quad\quad      X_{test}: \text{matrix with testing samples}, \\
&\quad\quad      y_{test}: \text{vector with testing labels}
):\\
\\
& \quad t := 0 \\
& \quad \text{compute initial population:  } \mathcal{B}_0 = (b_{1,0}, \dots, b_{m,0}) \\
& \quad \text{compute fitness for each individual } b_{i, 0}\\
& \quad \text{compute maximum fitness } \phi_0 \text{ for } \mathcal{B}_0\\
\\
& \quad \textbf{while  } \phi_t < \phi_m \textbf{ and } t < k: \\
& \quad \quad   \textbf{for  }i := 0 \textbf{ to } m - 1: \\
& \quad \quad  \quad     \text{select an individual  } b_{i,t+1} \text{ from  } \mathcal{B}_t \\
& \quad \quad   \textbf{for  } i := 0 \textbf{ to } m-2 \textbf{ step } 2: \\
& \quad \quad  \quad     \textbf{if } \text{Random}[0,1] \leq p_c: \\
& \quad \quad  \quad \quad         \text{cross } b_{i,t+1} \text{ with } b_{i+1,t+1} \\
& \quad \quad   \textbf{for  } i := 0 \textbf{ to } m -1: \\
& \quad \quad  \quad     \text{eventually mutate } b_{i,t+1} \\
& \quad \quad t := t + 1 \\
& \quad \quad \text{compute fitness for each individual } b_{i, t}\\
& \quad \quad \text{compute maximum fitness } \phi_t \text{ for } \mathcal{B}_t\\
& \quad\quad  \textbf{if } \phi_t - \phi_{t-1} < \phi_T:\\
&\quad\quad\quad \textbf{break}\\
&\\
& \quad \textbf{return } b_{i, t} \text{ with maximum } \phi \text{ and its fitness}
\end{aligned}
$$

where the sampling policy is: replace selected individuals with one of its children and all the others die immediately. The genetic operations are:
1. ***selection***: it can be deterministic, but in most implmenetations it has random components. The one used in this notebook is *"roulette wheel"* (or *"proportional selection"*) where the probability to choose a certain individual is proportional to its fitness:
$$P\{b_{i, t} \text{ is selcted}\} = \frac{\phi(b_{i, t})}{\sum_{k = 1}^m\phi(b_{k, t})}$$

It's called *"roulette"* because it resembles a roulette game where the slots aren't equally wide.

2. ***crossover***: it's the exchange of genes between chromosomes of the two parents. We can realize it using different techniques:
    * *single point*: the two vectors are cut at a randomly chosen position and the two tails are swapped
    * *multiple points crossover*: instead of only one, $n$ breaking points are randomly; then, every second section is swapped.
    $$
    \begin{aligned}
    &\textbf{crossover}(p_1:\text{first parent},\\
    &\quad\quad\quad\quad\quad p_2: \text{second parent}, \\
    &\quad\quad\quad\quad\quad n: \text{number of crossover points}):\\
    &\text{create two empty arrays $c_1$, $c_2$ for the children}\\
    &\text{generate $n$ distinct positions ($P$) in range $[0, m-1]$ and sort them}\\
    &\text{swap} := \text{false}\\
    &\text{start} := 0\\
    &\\
    &\textbf{for  } c \in P \cup [n]:\\
    &\quad \textbf{if } \text{swap}:\\
    &\quad\quad c_1[\text{start} : k] = p_2[\text{start} : k]\\
    &\quad\quad c_2[\text{start} : k] = p_1[\text{start} : k] \\  
    &\quad\textbf{else:}\\
    &\quad\quad c_1[\text{start} : k] = p_1[\text{start} : k]\\
    &\quad\quad c_2[\text{start} : k] = p_2[\text{start} : k]  \\
    &\quad\text{swap} := \textbf{not } \text{swap}    \\
    &\quad\text{start} := c\\
    &\\
    &\textbf{return } c_1, c_2
    \end{aligned}
    $$

3. ***mutation***: in reality, the probability that a certain gene is mutated is almost equal for all genes, so for the $i\text{-th}$ gene if $\text{random}[0, 1] \leq p_m$ we invert it.

In [2]:
import numpy as np

import sys
sys.path.append("..")
from utils.models.RandomForest import *

from concurrent.futures import ThreadPoolExecutor

In [3]:
def fitness(X_train, y_train, X_test, y_test):
    """
        Cmputes the fitness of a RF trained on the given features, by training and testing it.
        The used metric is the accuracy.

        Args:
            X_train, X_test (np.ndarray): the feature matrix for training and testing.
            y_train, y_test (np.ndarray): the target variable for training and testing.

        Returns:
            fitness: the accuracy value.
    """

    #A tradeoff between performance and model capabilities has been chosen: this might impact on the computed features vector;
    #see the nootebook on RF for a comparison and detailed explaination of how this code works.
    rf = RandomForest(n_trees = 10,
                      criterion = "gini", 
                      max_depth = 8, 
                      min_samples_split = 3, 
                      min_samples_leaf = 2, 
                      min_impurity_decrease = 0.0,
                      max_thresholds = 20, 
                      random_state = 0)
    
    rf.fit(X_train, y_train)

    y_pred, _ = rf.predict(X_test)

    #We could have used also model_evaluation.compute_metrics(...)["accuracy"]: to speed up, we'll use only this.
    tp = np.sum((y_pred == 1) & (y_test == 1))
    tn = np.sum((y_pred == 0) & (y_test == 0))

    return (tp + tn) / len(y_test)

In [4]:
def crossover(p1, p2, n):
    """
        This method computes the n-crossover between two parents by following the algorithm described in theory part.

        Args:
            p1 (np.ndarray): the chromosome of the first parent.
            p2 (np.ndarray): the chromosome of te second parent.
            n (int): number of split points.
        
        Returns:
            c1 (np.ndarray): the first children.
            c2 (np.ndarray): the second children.
    """

    c1 = np.empty(p1.shape)
    c2 = np.empty(p2.shape)
    crs_pts = np.empty(n, dtype = int)

    #Available positions are only the ones from 1 to len(p1) -1
    availables = np.array(range(1, len(p1) - 1)) 

    #crs_pts first index
    crs_pts[0] = np.random.choice(range(1, len(p1) - 1))

    for i in range(n - 1):
        #Update available positions by deleting the neighbor positions of the last extracted: what we want to avoid is a
        #situation like [2, 3] that means that no crossover of genes in 2 - 3 will be made.
        availables = np.setdiff1d(availables, np.array([crs_pts[i] - 1, crs_pts[i], crs_pts[i] + 1]))

        #Generate another index
        crs_pts[i + 1] = np.random.choice(availables)

    crs_pts = np.sort(crs_pts)
    
    #We need to append to crs_points the last index of the array, otherwise we wouldn't swap the last genes of the chromosomes!
    crs_pts = np.append(crs_pts, len(p1))

    swap = False
    start = 0

    for c in crs_pts:
        if swap:
            c1[start : c], c2[start : c] = p2[start : c], p1[start : c]
        else:
            c1[start : c], c2[start : c] = p1[start : c], p2[start : c]

        #If one chuck of genes has been swapped, the next should not be swapped and viceversa
        swap = not swap

        #The new start of the chuck becomes the endpoint of the previous
        start = c

    return c1, c2

In [5]:
def spawn_threads_fitness(B, X_train, y_train, X_test, y_test):
    """
        Each RF can be trained by a different thread to increase performance. 
        This methods creates a thread pool and assigns a features vector to a spwaned thred.

        Args:
            B (np.ndarray) : the array containing the selected features.
            X_train, X_test (np.ndarray): the feature matrix for training and testing.
            y_train, y_test (np.ndarray): the target variable for training and testing.
        
        Returns:
            fits: the list of fitness scores.
    """

    #Create a thread pool with at most 20 threads (typically max_workers = min(20, num-cores_of_machine))
    with ThreadPoolExecutor(max_workers = 20) as executor:
        #To vectorize the function fitness, we need to create a list of tuples, each containing the attributes to be passed to the method
        args = [(X_train[:, b.astype(bool)], y_train, X_test[:, b.astype(bool)], y_test) for b in B]
        
        #Then, we use the exeutor to compute the fitnes scores
        fits = list(executor.map(lambda p: fitness(*p), args)) 
        
        #When an executor has finished, we shutdown it
        executor.shutdown()
    
    return np.array(list(fits))

In [6]:
def genetic_algorithm(phi_m, m, F, k, p_m, p_c, n, threshold, X_train, y_train, X_test, y_test):
    """
        Genetic algorithm for feature selection.

        Args:
            phi_m (float): target fitness threshold.
            m (int): population size.
            F (list): feature set.
            k (int): maximum generations.
            p_m (float): mutation probability.
            p_c (float): crossover probability.
            n (int): number of crossover points.
            threshold (float): threshold on the increment of fitness.
            X_train, y_train, X_test, y_test (np.ndarray): training and test data.
        
        Returns:
            features (np.ndarray): the selected features.
            max_fit (float): the fitness value of the solution.
            feats_history (list): the history of the feature vectors with highest fitness.
            fits_history (list): the history of fitness values.
    """

    np.random.seed(0)
    t = 0

    #Compute the initial population
    B = np.random.choice([0, 1], size = (m, len(F)))

    #Compute the fitness of each individual in the initial population and the maximum fit reached
    fits = spawn_threads_fitness(B, X_train, y_train, X_test, y_test)
    max_fit, max_fit_id = np.max(fits), np.argmax(fits)

    feats_history = [B[max_fit_id, :]]
    fitness_history = [max_fit]

    #for i in range(m):
    #    print(f"{B[i, :]}   {fits[i]}")

    while max_fit < phi_m and t < k:
        print(f"Currect generation: {t}; current fitness: {max_fit}")
        
        #Start with selection
        #Define the "roulette wheel" probabilities (note: in this case all fitness measures are positive)
        roulette_probs = fits / np.sum(fits)

        #Choose randomly m individuals: by how selection probabilities are defined, individuals with higher fitness should be selected more more frequently
        ids = np.random.choice(range(m), size = m, replace = True, p = roulette_probs)

        #Update the population with the selected individuals
        B = B[ids, :]

        #Then, apply crossover
        for i in range(0, m - 1):
            #Crossover is to be applied only with p_c probability
            if np.random.rand() < p_c:
                B[i], B[i + 1] = crossover(B[i], B[i + 1], n)

        #Finally, apply mutation
        for i in range(m):
            mut_probs = np.random.rand(len(F))

            #Mutation has to be applied only when the probability on the sigle gene is < p_m
            #As defined in theory, it inverts the bit
            B[i, mut_probs < p_m] = 1 - B[i, mut_probs < p_m]

        #At the end, compute again the fitness of each individual in the initial population and the maximum fit reached
        fits = spawn_threads_fitness(B, X_train, y_train, X_test, y_test)
        max_fit, max_fit_id = np.max(fits), np.argmax(fits)
        
        #Terminating condition
        if max_fit - fitness_history[-1] < threshold:
            break

        feats_history.append(B[max_fit_id, :])
        fitness_history.append(max_fit)

        t = t + 1

    return B[max_fit_id], max_fit, feats_history, fitness_history

### ***Usage of the GA***

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

from utils.preprocessing import undersample

In [8]:
#Load the dataset
data = pd.read_csv("../creditcard_2021.csv")

#"Class" column is the target variable, so we remove it from the feature matrix and store it in the variable y
X = data.drop(columns = ["Class"])
y = data["Class"]

X = np.array(X)
y = np.array(y)

In [9]:
#Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train, y_train = undersample(X_train, y_train, 0, 1 - (sum(y_train) * 300.0 / X_train.shape[0]))

In [11]:
best_feats, max_fit, feats_hist, fit_hist = genetic_algorithm(1.0, 90, range(X.shape[1]), 100, 0.01, 0.85, 2, 1e-8, X_train, y_train, X_test, y_test)

Currect generation: 0; current fitness: 0.9994967405170698
Currect generation: 1; current fitness: 0.9995318516437859


In [12]:
best_feats

array([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0])

In [23]:
def decoder(c):
    str_version = []

    for i in range(len(c)):
        if c[i] == 1:
            str_version.append(f"V{i}")
    
    for i in range(len(c)):
        if c[i] == "V0":
            c[i] = "Time"
        
        if c[i] == "V29":
            c[i] = "Amount"

    return str_version

print(decoder(best_feats))

['V2', 'V5', 'V7', 'V10', 'V12', 'V14', 'V16', 'V17', 'V19', 'V21', 'V24', 'V28']


In [24]:
import utils.model_evaluation as me

for name in me.feature_vectors.keys():
    print(f"{name}: {me.feature_vectors[name]}")

v1: ['V1', 'V5', 'V7', 'V8', 'V11', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'Amount']
v2: ['V1', 'V6', 'V13', 'V16', 'V17', 'V16', 'V17', 'V22', 'V23', 'V28', 'Amount']
v3: ['V2', 'V11', 'V12', 'V13', 'V15', 'V16', 'V17', 'V18', 'V20', 'V21', 'V24', 'V26', 'Amount']
v4: ['V2', 'V7', 'V10', 'V13', 'V15', 'V17', 'V19', 'V28', 'V17', 'Amount']
v5: ['Time', 'V1', 'V7', 'V8', 'V9', 'V11', 'V12', 'V14', 'V15', 'V22', 'V27', 'V28', 'Amount']
v6: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
v7: ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']


In [ ]:
#The results shw that the GA is working as expected, but it's not finding any features vector contained into me.feature_vectors. Still, the result appears to be similar
#to v4 ('V2', 'V7', 'V10', 'V17', 'V19', 'V28') and v1 ('V5', 'V7', 'V14', 'V16', 'V17', 'V19', 'V21', 'V24')